In [1]:
sheet_url = "https://docs.google.com/spreadsheets/d/1aw6U3K2ZdgjniBQ5ei6F4IGaAz8d5uuG/edit#gid=66557950"

año = '2023'

In [2]:
#Transformación de Data
import pandas as pd
import numpy as np
import mysql.connector as connection
import warnings
warnings.filterwarnings('ignore')


url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url_1)

#df = df[["FECHA", "PERIODO","TIPO ACOMPAÑAMIENTO","RESPONSABLES DE COORD CON LA UO","UO (LÍDERA)","ARTICULACIÓN CON OTRAS UO","HORA","DIRECTIVOS/JEFES (SI/NO)","TIPO","COD DE AT","COD SISGED","TITULO", "ALCANCE","SESIONES","REGIONES POR FECHA","REGIONES DE LA ATA","ENFASIS TÉMATICO DE LA AT","TIPO DE AT (VIRTUAL/ PRESENCIAL)","DRE/GRE, UGEL","PERFIL","NUDO","OBJETIVO","PRODUCTO","PROCESOS","PROCESOS 0","PROCESOS 1","NORMA RELACIONADA"]]
df = df[["FECHA", "PERIODO","TIPO ACOMPAÑAMIENTO","RESPONSABLES DE COORD CON LA UO","UO (LÍDERA)","ARTICULACIÓN CON OTRAS UO","HORA","DIRECTIVOS/JEFES (SI/NO)","TIPO","COD DE AT","COD SISGED","TITULO", "ALCANCE","SESIONES","REGIONES POR FECHA","REGIONES DE LA ATA","ENFASIS TÉMATICO DE LA AT","TIPO DE AT (VIRTUAL/ PRESENCIAL)","DRE/GRE, UGEL","PERFIL","NUDO","OBJETIVO","PRODUCTO","PROCESOS","PROCESOS 0","PROCESOS 1","NORMA RELACIONADA", "TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)", "BIAE 2023"]]

df["PERFILX"] = df["PERFIL"]
#PARCHE TIPO ACOMP
#df["TIPO ACOMPAÑAMIENTO"] = np.where(df["COD SISGED"] == "ATE03873" , "MIXTA", df["TIPO ACOMPAÑAMIENTO"])

# Limpieza sin fecha
df["FECHA"].fillna('', inplace=True)
df = df.drop(df[df["FECHA"]==""].index)
df["COD SISGED"].fillna('', inplace=True)
df["COD DE AT"].fillna('', inplace=True)
df["SESIONES"].fillna('0', inplace=True)
df["SESIONES"] = df["SESIONES"].astype(int)
df["BIAE 2023"].fillna('', inplace=True)
df1 = df[df["TIPO"]=="AT"]
df3 = df1.drop(df1[df1["COD SISGED"]==""].index)
df3 = df3.drop(['TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)'], axis=1)
df3["kpi_titulo"] = df3["COD SISGED"].map(str)+df3["TITULO"].map(str)
df3["kpi_biae"] = df3["COD SISGED"].map(str)+df3["BIAE 2023"].map(str)
df3["kpi_codat"] = df3["COD SISGED"].map(str)+df3["COD DE AT"].map(str)
df3["kpi_3"] = df3["COD SISGED"].map(str)+df3["TIPO ACOMPAÑAMIENTO"].map(str)
df3["kpi_4"] = df3["COD SISGED"].map(str)+df3["UO (LÍDERA)"].map(str)
df3["kpi_5"] = df3["COD SISGED"].map(str)+df3["ENFASIS TÉMATICO DE LA AT"].map(str)
df3["kpi_6"] = df3["COD SISGED"].map(str)+df3["NUDO"].map(str)
df3["kpi_7"] = df3["COD SISGED"].map(str)+df3["OBJETIVO"].map(str)
df3["kpi_8"] = df3["COD SISGED"].map(str)+df3["PROCESOS"].map(str)
df3["kpi_9"] = df3["COD SISGED"].map(str)+df3["PROCESOS 0"].map(str)
df3["kpi_10"] = df3["COD SISGED"].map(str)+df3["PROCESOS 1"].map(str)
df3["kpi_11"] = df3["COD SISGED"].map(str)+df3["ALCANCE"].map(str)
df3["kpi_12"] = df3["COD SISGED"].map(str)+df3["PERFILX"].map(str)
df3["kpi_13"] = df3["COD SISGED"].map(str)+df3["PRODUCTO"].map(str)
df3["kpi_14"] = df3["COD SISGED"].map(str)+df3["DIRECTIVOS/JEFES (SI/NO)"].map(str)
df3["kpi_15"] = df3["COD SISGED"].map(str)+df3["ARTICULACIÓN CON OTRAS UO"].map(str)
#df3

In [3]:
#Crear DataFrame para SQL
df3["FECHA1"] = df["FECHA"].str.replace('/','')
df3["FECHA1"]= df3['FECHA1'].astype(str).str.zfill(8)
df3["FECHA2"] = pd.to_datetime(df3["FECHA1"],format='%d%m%Y')

##GROUPBY BUEN EJEMPLO
df4 = pd.DataFrame(df3.groupby(["COD SISGED"])["FECHA2"].max())
##PARA CONVERTIRLO EN LLAVE, LO ENVÍA ADELANTE CON .INDEX
df4['COD SISGED'] = df4.index
df4["llave1"] = df4['COD SISGED']

df3["TITULO"]=df3["TITULO"].str.upper()
df3["BIAE 2023"]=df3["BIAE 2023"].str.upper()
df3["ALCANCE"]=df3["ALCANCE"].str.upper()
df3["ARTICULACIÓN CON OTRAS UO"]=df3["ARTICULACIÓN CON OTRAS UO"].str.upper()
df3["ENFASIS TÉMATICO DE LA AT"]=df3["ENFASIS TÉMATICO DE LA AT"].str.upper()
df3["PERFILX"]=df3["PERFILX"].str.upper()
df3["NUDO"]=df3["NUDO"].str.upper()
df3["OBJETIVO"]=df3["OBJETIVO"].str.upper()
df3["PRODUCTO"]=df3["PRODUCTO"].str.upper()
df3["PROCESOS"]=df3["PROCESOS"].str.upper()
df3["PROCESOS 0"]=df3["PROCESOS 0"].str.upper()
df3["PROCESOS 1"]=df3["PROCESOS 1"].str.upper()

df5=df3.drop(['FECHA'], axis=1)
df5=df5.drop(['PERIODO'], axis=1)
df5=df5.drop(['RESPONSABLES DE COORD CON LA UO'], axis=1)
df5=df5.drop(['REGIONES POR FECHA'], axis=1)
df5=df5.drop(['REGIONES DE LA ATA'], axis=1)
df5=df5.drop(['SESIONES'], axis=1)
df5=df5.drop(['TIPO DE AT (VIRTUAL/ PRESENCIAL)'], axis=1)
df5=df5.drop(['DRE/GRE, UGEL'], axis=1)
df5=df5.drop(['NORMA RELACIONADA'], axis=1)
df5=df5.drop(['PERFIL'], axis=1)
#df5=df5.drop(['TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)'], axis=1)
df5=df5.drop(['kpi_titulo'], axis=1)
df5=df5.drop(['kpi_biae'], axis=1)
df5=df5.drop(['kpi_codat'], axis=1)
df5=df5.drop(['kpi_3'], axis=1)
df5=df5.drop(['kpi_4'], axis=1)
df5=df5.drop(['kpi_5'], axis=1)
df5=df5.drop(['kpi_6'], axis=1)
df5=df5.drop(['kpi_7'], axis=1)
df5=df5.drop(['kpi_8'], axis=1)
df5=df5.drop(['kpi_9'], axis=1)
df5=df5.drop(['kpi_10'], axis=1)
df5=df5.drop(['kpi_11'], axis=1)
df5=df5.drop(['kpi_12'], axis=1)
df5=df5.drop(['kpi_13'], axis=1)
df5=df5.drop(['kpi_14'], axis=1)
df5=df5.drop(['kpi_15'], axis=1)
df5=df5.drop(['FECHA1'], axis=1)
df5=df5.drop(['FECHA2'], axis=1)
df5=df5.drop(['HORA'], axis=1)

df6= df5.drop_duplicates()
df6["llave1"] = df6["COD SISGED"]
##LLAMA A LA ÚLTIMA FECHA CON BUSCAR V DE df4
df6 = df6.merge(df4, on="llave1", how="left")
df6['AÑO'] = pd.DatetimeIndex(df6['FECHA2']).year
##PARA OBTENER FORMATO "01. ENE"
df6['MES'] = pd.DatetimeIndex(df6['FECHA2']).month
df6['MES2'] = pd.to_datetime(df6['MES'], format='%m').dt.month_name(locale = 'Spanish').str.slice(stop=3)
df6["MES"]= df6['MES'].astype(str).str.zfill(2)
df6['MESX'] = df6["MES"].map(str)+". "+df6["MES2"].map(str)

df6=df6.drop(['MES'], axis=1)
df6=df6.drop(['MES2'], axis=1)

df6["FECHA2"] = pd.to_datetime(df6["FECHA2"]).dt.date
df6['FECHA2'] = df6['FECHA2'].astype(str)
df6["AÑO"] = df6["AÑO"].astype(str)
df6.fillna('', inplace=True)
df6["COD SISGED_x"] = df6["COD SISGED_x"].str.replace(' ','')
import openpyxl
df6.to_excel("C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Planificador.xlsx", index=False, sheet_name='base_datos')

#Crear DataFrame para SQL2
df7 = df3
df7['HORA'] = df7['HORA'].str.replace('.',':')
df7['HORA'] = df7['HORA'].str.replace(' ','')
df7['HORA'] = df7['HORA'].str.replace(';',',')
df7["SESIONES"].fillna('0', inplace=True)
df7['SESIONES']="S"+df7["SESIONES"].map(str)
df7['SESIONES']=df7['SESIONES'].str.replace(',',',S')
df = df.drop(df[df["FECHA"]==""].index)
df7['REGIONES POR FECHA']=df7['REGIONES POR FECHA'].str.upper()

df8 = df7[["COD SISGED","FECHA2","HORA", "SESIONES", "REGIONES POR FECHA", "DIRECTIVOS/JEFES (SI/NO)",  "ALCANCE","TIPO DE AT (VIRTUAL/ PRESENCIAL)"]].sort_values(by='FECHA2', ascending=False)

df8.fillna('', inplace=True)

df8["FECHA2"] = pd.to_datetime(df8["FECHA2"]).dt.date
df8['FECHA2'] = df8['FECHA2'].astype(str)

df8.to_excel("C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Calendario.xlsx", index=False, sheet_name='base_datos')
#df5

In [4]:
df6.info()
#df8

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 150
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TIPO ACOMPAÑAMIENTO        151 non-null    object
 1   UO (LÍDERA)                151 non-null    object
 2   ARTICULACIÓN CON OTRAS UO  151 non-null    object
 3   DIRECTIVOS/JEFES (SI/NO)   151 non-null    object
 4   TIPO                       151 non-null    object
 5   COD DE AT                  151 non-null    object
 6   COD SISGED_x               151 non-null    object
 7   TITULO                     151 non-null    object
 8   ALCANCE                    151 non-null    object
 9   ENFASIS TÉMATICO DE LA AT  151 non-null    object
 10  NUDO                       151 non-null    object
 11  OBJETIVO                   151 non-null    object
 12  PRODUCTO                   151 non-null    object
 13  PROCESOS                   151 non-null    object
 14  PROCESOS 0

In [5]:
#Delete Planificador SISGED
import Conexion_mysql as cnx

sql1="delete from ATA_PLANIFICADOR_SISGED where año = "+ año
cnx.query(sql1)

DataError: 1292 (22007): Truncated incorrect DOUBLE value: 'ATE03908'

In [ ]:
#Insert Planificador SISGED
sql2 = ''
for (rows,rs) in df6.iterrows():
                i1 = rs[6]
                i2 = rs[5]
                i3 = rs[0]
                i4 = rs[1] #uo
                i5 = rs[7] #titulo
                i6 = rs[16] #BIAE    BIAE
                i7 = rs[9] #enfasis    ENFASIS TEMATICO
                i8 = rs[10] #nudo   NUDO
                i9 = rs[11] #objetivo   OBJETIVO
                i10 = rs[13] #procesos  PROCESOS
                i11 = rs[14] #procesos_0    PROCESOS 0
                i12 = rs[15] #procesos_1 PROCESOS 1
                i13 = rs[8] #alcance   ALCANCE
                i14 = rs[17] #perfil    PERFILX
                i15 = rs[12] #producto   PRODUCTO
                i16 = rs[3] #directivos    DIRECTIVOS/JEFES (SI/NO)
                i17 = rs[20] #año   AÑO
                i18 = rs[22] #mes   MESX
                text = "('"+i1+"', '"+i2+"', '"+i3+"', '"+i4+"', '"+i5+"', '"+i6+"', '"+i7+"', '"+i8+"', '"+i9+"', '"+i10+"', '"+i11+"', '"+i12+"', '"+i13+"', '"+i14+"', '"+i15+"', '"+i16+"', '"+i17+"', '"+i18+"'),"
                sql2 = sql2 + text
sql2=sql2[:-1]
sql3="insert into ata_planificador_sisged values " + sql2
cnx.query(sql3)

In [ ]:
#Delete Planificador Sesiones
sql4="delete from ATA_PLANIFICADOR_SESION where year(fecha)='" + año + "'"
cnx.query(sql4)

In [ ]:
#Insert Planificador Sesiones
sql5=''
for (rows,rs) in df8.iterrows():
                i1 = rs[0]
                i2 = rs[1]
                i3 = rs[2]
                i4 = rs[3]
                i5 = rs[4]
                i6 = rs[5]
                i7 = rs[6]
                i8 = rs[7]
                text = "('"+i1+"', '"+i2+"', '"+i3+"', '"+i4+"', '"+i5+"', '"+i6+"', '"+i7+"', '"+i8+"'),"
                sql5=sql5 + text
sql5=sql5[:-1]
sql5="insert into ATA_PLANIFICADOR_SESION (COD_SISGED, FECHA, HORA, SESION, REGION, DIRECTIVO, ALCANCE, virtual_presc) values " + sql5
cnx.query(sql5)

In [ ]:
df9 = df[df["TIPO"]=="AT"]

#df = df[["FECHA", "PERIODO","TIPO ACOMPAÑAMIENTO","RESPONSABLES DE COORD CON LA UO","UO (LÍDERA)","ARTICULACIÓN CON OTRAS UO","HORA","DIRECTIVOS/JEFES (SI/NO)",
#   "TIPO","COD DE AT", "COD SISGED","TITULO", "ALCANCE","SESIONES","REGIONES POR FECHA","REGIONES DE LA ATA","ENFASIS TÉMATICO DE LA AT","TIPO DE AT (VIRTUAL/ PRESENCIAL)",
#   "DRE/GRE, UGEL","PERFIL","NUDO", "OBJETIVO","PRODUCTO","PROCESOS","PROCESOS 0","PROCESOS 1","NORMA RELACIONADA"]]


df9=df9.drop(['FECHA'], axis=1) #FECHA
df9=df9.drop(['PERIODO'], axis=1) #PERIODO 
df9=df9.drop(['TIPO ACOMPAÑAMIENTO'], axis=1) #TIPO
df9=df9.drop(['RESPONSABLES DE COORD CON LA UO'], axis=1) #RESPONSABLES
df9=df9.drop(['ARTICULACIÓN CON OTRAS UO'], axis=1) #ARTICULACIÓN
df9=df9.drop(['HORA'], axis=1) #HORA
df9=df9.drop(['DIRECTIVOS/JEFES (SI/NO)'], axis=1) #DIRECTIVOS 
df9=df9.drop(['BIAE 2023'], axis=1) #BIAE 2023 
df9=df9.drop(['ALCANCE'], axis=1) #ALCANCE 
df9=df9.drop(['SESIONES'], axis=1) #SESIONES 
df9=df9.drop(['REGIONES POR FECHA'], axis=1) #REGIONES
df9=df9.drop(['REGIONES DE LA ATA'], axis=1) #REGIONES
df9=df9.drop(['TIPO DE AT (VIRTUAL/ PRESENCIAL)'], axis=1) #TIPO
df9=df9.drop(['DRE/GRE, UGEL'], axis=1) #DRE
df9=df9.drop(['PERFIL'], axis=1) #PERFIL
df9=df9.drop(['PERFILX'], axis=1) #PERFILX
df9=df9.drop(['NUDO'], axis=1) #NUDO
df9=df9.drop(['OBJETIVO'], axis=1) #OBJETIVO
df9=df9.drop(['PRODUCTO'], axis=1) #PRODUCTO
df9=df9.drop(['PROCESOS'], axis=1) #PROCESOS
df9=df9.drop(['PROCESOS 0'], axis=1) #PROCESOS 0
df9=df9.drop(['PROCESOS 1'], axis=1) #PROCESOS 1
df9=df9.drop(['NORMA RELACIONADA'], axis=1) #NORMA


df9["TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)"].fillna('', inplace=True)
df9 = df9.drop(df9[df9["TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)"]==""].index)

df9["ENFASIS TÉMATICO DE LA AT"].fillna('', inplace=True)

df9["COD SISGED"].fillna('', inplace=True)
df9 = df9.drop(df9[df9["COD SISGED"]==""].index)

df9['CONCAT'] = df9['COD SISGED'].map(str)+df9['TEMA (OFERTA / DEMANDA / TEMÁTICA GENERAL)'].map(str)
df9 = df9.drop_duplicates(subset=['CONCAT'])

##LLAMA A LA ÚLTIMA FECHA CON BUSCAR V DE df4
df9['llave1'] = df9['COD SISGED']
df9 = df9.merge(df4, on="llave1", how="left")

df9['AÑO'] = pd.DatetimeIndex(df9['FECHA2']).year
##PARA OBTENER FORMATO "01. ENE"
df9['MES'] = pd.DatetimeIndex(df9['FECHA2']).month
df9['MES2'] = pd.to_datetime(df9['MES'], format='%m').dt.month_name(locale = 'Spanish').str.slice(stop=3)
df9["MES"]= df9['MES'].astype(str).str.zfill(2)
df9['MESX'] = df9["MES"].map(str)+". "+df9["MES2"].map(str)

df9=df9.drop(['CONCAT'], axis=1) #CONCAT
df9=df9.drop(['llave1'], axis=1) #llave1
df9=df9.drop(['COD SISGED_y'], axis=1) #COD SISGED_y
df9=df9.drop(['MES'], axis=1) #MES
df9=df9.drop(['MES2'], axis=1) #MES2

df9['AÑO'] = df9['AÑO'].astype(str)
df9['FECHA2'] = df9['FECHA2'].astype(str)

df9.to_excel("C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Tema.xlsx", index=False, sheet_name='base_datos')
#df9

PermissionError: [Errno 13] Permission denied: 'C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Tema.xlsx'

In [ ]:
#Delete Planificador Sesiones
sql6=''
sql6="delete from ATA_TEMA where año='" + año + "'"
cnx.query(sql6)

In [ ]:
host="localhost"
database ='difoca'
user="root"
passwd="root1"

try:
    con = connection.connect(host=host, database =database,user=user, passwd=passwd,use_pure=True)
    cur=con.cursor()

    for (rows,rs) in df9.iterrows():
        x1 = rs[8] #AÑO
        x2 = rs[9] #MESX
        x3 = rs[7] #FECHA2
        x4 = rs[6] #TEMA
        x5 = rs[2] #COD DE ATA
        x6 = rs[3] #COD_SISGED_x
        x7 = rs[4] #TITULO
        x8 = rs[0] #UO
        x9 = rs[5] #ENFASIS TEMÁTICO DE LA ATA
        query = "insert into ATA_TEMA (año, periodo, fecha, tema, cod_ata, cod_sisged, titulo, uo, enfasis) VALUES ('"+x1+"', '"+x2+"', '"+x3+"', '"+x4+"', '"+x5+"', '"+x6+"', '"+x7+"', '"+x8+"', '"+x9+"')"
        cur.execute(query)
    con.commit()
    con.close()
except Exception as e:
    con.close()
    print(str(e))

In [ ]:
try:
    con = connection.connect(host=host, database =database,user=user, passwd=passwd,use_pure=True)
    cur=con.cursor()
    query = "delete from ata_tema_ma where año = '"+año+ "'"
    cur.execute(query)
    con.commit()
    con.close()
except Exception as e:
    con.close()
    print(str(e))

In [ ]:
carpeta_ata = "C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)"
excel_1 = 'Reporte_ATA_Tema_ma'
df10 = pd.read_excel(carpeta_ata+"/"+excel_1+".xlsx", sheet_name='base_datos')
df10["VALOR"]=df10["VALOR"].astype(str)
try:
    con = connection.connect(host=host, database =database,user=user, passwd=passwd,use_pure=True)
    cur=con.cursor()

    for (rows,rs) in df10.iterrows():
        x1 = rs[0] #Tema
        x2 = rs[1] #Considera
        x3 = año
        x4 = rs[3]
        query = "insert into ata_tema_ma (tema, considera, año, valor) VALUES ('"+x1+"', '"+x2+"', '"+x3+"', '"+x4+"')"
        cur.execute(query)
    con.commit()
    con.close()
except Exception as e:
    con.close()
    print(str(e))